In [1]:
import os
import glob
import subprocess
import json
from pathlib import Path
from midiutil import MIDIFile

# CONFIGURACIÓN DE PARÁMETROS

In [2]:
MIDI_DIR = "patterns"  # Carpeta donde se guardarán los archivos MIDI
audio_dir = "wav"  # Carpeta donde se guardarán los archivos WAV
jams_dir = "jams"  # Carpeta donde se guardarán los archivos JAMS

In [3]:
def ensure_directories_exist():
    for directory in [MIDI_DIR, audio_dir, jams_dir]:
        Path(directory).mkdir(parents=True, exist_ok=True)

ensure_directories_exist()

# FUNCIÓN PARA GENERAR ACORDES

In [4]:
def create_chord(base, offsets, inversion=0):
    """Genera un acorde en formato de lista de notas MIDI."""
    notes = [base + offset for offset in offsets]
    for _ in range(inversion):
        notes.append(notes.pop(0) + 12)  # Subir la primera nota una octava
    return notes

# FUNCIÓN PARA CREAR ARCHIVO MIDI

In [5]:
def create_midi_file(chord, filename, style="chords"):
    """Genera un archivo MIDI con el acorde dado."""
    track = 0
    channel = 0
    time = 0  # En beats
    duration = 2  # En beats
    tempo = 60  # BPM
    volume = 100  # 0-127

    MyMIDI = MIDIFile(1)  # Un solo track
    MyMIDI.addTempo(track, time, tempo)

    for note in chord:
        MyMIDI.addNote(track, channel, note, time, duration, volume)
    
    midi_path = os.path.join(MIDI_DIR, style, filename + ".mid")
    Path(os.path.dirname(midi_path)).mkdir(parents=True, exist_ok=True)
    
    with open(midi_path, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    
    return midi_path

# FUNCIÓN PARA CONVERTIR MIDI A WAV

In [6]:
def convert_midi_to_wav(midi_path):
    """Convierte un archivo MIDI a WAV usando Timidity."""
    wav_path = midi_path.replace(MIDI_DIR, audio_dir).replace(".mid", ".wav")
    Path(os.path.dirname(wav_path)).mkdir(parents=True, exist_ok=True)
    
    subprocess.call(['timidity', midi_path, '-Ow1', '-s', '16kHz', '-o', wav_path])
    return wav_path

# FUNCIÓN PARA GENERAR ARCHIVO JAMS

In [7]:
def create_jams_file(midi_path, chord_labels):
    """Crea un archivo JAMS con las etiquetas de los acordes."""
    jams_path = midi_path.replace(MIDI_DIR, jams_dir).replace(".mid", ".jams")
    Path(os.path.dirname(jams_path)).mkdir(parents=True, exist_ok=True)
    
    jams_data = {
        "chord": [
            {"start": i * 2, "end": (i + 1) * 2, "label": chord}
            for i, chord in enumerate(chord_labels)
        ]
    }
    
    with open(jams_path, "w") as f:
        json.dump(jams_data, f, indent=4)
    
    return jams_path

# FUNCIÓN PRINCIPAL PARA GENERAR PROGRESIÓN

In [8]:
def generate_chord_progression(progression_name, base_note, progression):
    """Genera una progresión de acordes en MIDI, WAV y JAMS."""
    chords = []
    labels = []
    base_midi = 48  # C4 en MIDI
    
    for numeral, offsets in progression.items():
        chord = create_chord(base_midi, offsets)
        filename = f"{base_note}-{numeral}-{progression_name}"
        midi_path = create_midi_file(chord, filename)
        convert_midi_to_wav(midi_path)
        chords.append(chord)
        labels.append(numeral)
    
    create_jams_file(midi_path, labels)
    print(f"Progresión {progression_name} generada con éxito!")

# EJEMPLO DE USO

In [9]:
example_progression = {
    "I": [0, 4, 7],
    "IV": [0, 5, 9],
    "V": [0, 7, 11],
    "I": [0, 4, 7]
}

generate_chord_progression("basic_progression", "C", example_progression)

Playing patterns/chords/C-I-basic_progression.mid
MIDI file: patterns/chords/C-I-basic_progression.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~6 seconds
Notes cut: 0
Notes lost totally: 0
Playing patterns/chords/C-IV-basic_progression.mid
MIDI file: patterns/chords/C-IV-basic_progression.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~6 seconds
Notes cut: 0
Notes lost totally: 0
Playing patterns/chords/C-V-basic_progression.mid
MIDI file: patterns/chords/C-V-basic_progression.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~6 seconds
Notes cut: 0
Notes lost totally: 0
Progresión basic_progression generada con éxito!
